In [1]:
#@title Drive Mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Final_Defence

/content/drive/MyDrive/Final_Defence


In [3]:
# Ultralytics YOLOv5 official repo (public)
!git clone https://github.com/ultralytics/yolov5.git



Cloning into 'yolov5'...
remote: Enumerating objects: 17564, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 17564 (delta 35), reused 7 (delta 7), pack-reused 17508 (from 4)
Receiving objects: 100% (17564/17564), 16.64 MiB | 16.29 MiB/s, done.
Resolving deltas: 100% (12029/12029), done.


In [4]:
%cd yolov5

/content/yolov5


In [5]:
# requirements.txt আছে repo তে; CUDA সংস্করণের সাথে PyTorch আলাদা ইনস্টল করতে হতে পারে
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.1 MB/s eta 0:00:00


In [ ]:
import os
# main directory যেখানে আপনি সব run results রাখবেন
PROJECT_DIR = '/content/drive/MyDrive/Final_Defence/yolov5'
os.makedirs(PROJECT_DIR, exist_ok=True)

In [ ]:
# dataset yaml (তোমার পূর্বের path)
DATA_YAML = '/content/drive/MyDrive/Final_Defence/data/dataset.yaml'

In [ ]:
# ট্রেনিং কনফিগারে পরিবর্তন করতে চাওলে নাম/epoch/batch এখানে ঠিক করো
EXPERIMENT_NAME = 'train_results'   # runs/train/<EXPERIMENT_NAME> হবে সরাসরি না, yolo5 auto name দেয়; আমরা project path দেব
EPOCHS = 100
IMG_SIZE = 640
BATCH = 16
DEVICE = 0   # GPU index; GPU না থাকলে 'cpu'

In [ ]:
# 4️⃣ Quick GPU check (optional)
# ================================
import torch
print("torch.cuda.is_available():", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
else:
    print("GPU not available. Use device='cpu' in training command (very slow).")

torch.cuda.is_available(): True
GPU name: Tesla T4


In [ ]:
!python train.py \
  --img 640 \
  --batch 16 \
  --epochs 100 \
  --data /content/drive/MyDrive/Final_Defence/data/dataset.yaml \
  --weights yolov5s.pt \
  --project /content/drive/MyDrive/Final_Defence/yolov5 \
  --name train_results


Streaming output truncated to the last 5000 lines.
      65/99      4.24G    0.02954    0.01326  0.0003748         37        640:  46% 32/69 [01:52<02:00,  3.26s/it]/content/drive/MyDrive/Final_Defence/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      65/99      4.24G     0.0294    0.01305  0.0003724         22        640:  48% 33/69 [01:59<02:31,  4.20s/it]/content/drive/MyDrive/Final_Defence/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      65/99      4.24G    0.02919    0.01291  0.0003681         38        640:  49% 34/69 [01:59<01:45,  3.02s/it]/content/drive/MyDrive/Final_Defence/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)`

In [6]:
from ultralytics import YOLO
import os

# -----------------------------
# Paths
# -----------------------------
MODEL_PATH = '/content/drive/MyDrive/Final_Defence/yolov5/train_results5/weights/best.pt'
DATA_YAML  = '/content/drive/MyDrive/Final_Defence/data/dataset.yaml'
RESULT_DIR = '/content/drive/MyDrive/Final_Defence/yolov5'

os.makedirs(RESULT_DIR, exist_ok=True)

# -----------------------------
# Load trained YOLO model
# -----------------------------
model = YOLO(MODEL_PATH)

# -----------------------------
# Test set evaluation with custom save path
# -----------------------------
results = model.val(
    data=DATA_YAML,
    split='test',
    save=True,        # save prediction images
    save_txt=True,    # save predicted bounding boxes
    save_conf=True,   # save confidence scores
    project=RESULT_DIR,  # custom folder
    name='test_results'  # subfolder inside RESULT_DIR
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.191 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)


TypeError: BaseModel.fuse() got an unexpected keyword argument 'verbose'

In [9]:
# -----------------------------
# Predict with YOLOv5
# -----------------------------
pred_labels = []

# Make predictions
results = model(image_paths)  # Just call model without verbose or extra arguments

for i, res in enumerate(results):
    if len(res.boxes) > 0:
        top_pred = res.boxes.cls[0].item()  # predicted class
        pred_labels.append(int(top_pred))
    else:
        pred_labels.append(-1)  # No detection



Ultralytics 8.3.192 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)


Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs


TypeError: DetectionModel.forward() got an unexpected keyword argument 'embed'

In [8]:
# Install ultralytics if not already
# !pip install ultralytics

import torch
from ultralytics import YOLO
import glob
import os
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# -----------------------------
# Load trained YOLOv5 model
# -----------------------------
model_path = "/content/drive/MyDrive/Final_Defence/yolov5/train_results5/weights/best.pt"
model = YOLO(model_path)

# -----------------------------
# Prepare test/validation images
# -----------------------------
test_folder = "/content/drive/MyDrive/Final_Defence/data/images/val"
all_images = glob.glob(os.path.join(test_folder, "*.jpg"))
all_images += glob.glob(os.path.join(test_folder, "*.png"))

# Select only first 205 images
image_paths = all_images[:205]
print(f"Total test images used: {len(image_paths)}")

# -----------------------------
# True labels
# -----------------------------
# Assuming your labels are in YOLO format in same structure as /val/labels/*.txt
# Classes: 0 = Normal, 1 = Lytic, 2 = Blastic
label_folder = "/content/drive/MyDrive/Final_Defence/data/labels/val"

true_labels = []

for img_path in image_paths:
    img_name = os.path.basename(img_path).replace(".jpg", ".txt").replace(".png", ".txt")
    label_path = os.path.join(label_folder, img_name)
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            lines = f.readlines()
            if len(lines) > 0:
                cls = int(lines[0].split()[0])
                true_labels.append(cls)
            else:
                true_labels.append(-1)  # No object
    else:
        true_labels.append(-1)

# -----------------------------
# Predict with YOLOv5
# -----------------------------
pred_labels = []

results = model(image_paths)  # Returns a list of Results objects

for i, res in enumerate(results):
    if len(res.boxes) > 0:
        top_pred = res.boxes.cls[0].item()  # predicted class
        pred_labels.append(int(top_pred))
    else:
        pred_labels.append(-1)  # No detection

# -----------------------------
# Compute confusion matrix
# -----------------------------
labels_to_use = [0, 1, 2]  # Normal, Lytic, Blastic
cm = confusion_matrix(true_labels, pred_labels, labels=labels_to_use)
print("Confusion Matrix:")
print(cm)



# -----------------------------
# Classification Report
# -----------------------------
report = classification_report(true_labels, pred_labels, labels=labels_to_use, target_names=["Blastic", "Lytic", "Normal"])
print("Classification Report:\n", report)

# -----------------------------
# Plot confusion matrix
# -----------------------------
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Blastic","Lytic","Normal"], yticklabels=["Blastic","Lytic","Normal"])
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.title("YOLOv5 Confusion Matrix")
plt.show()


Total test images used: 205


TypeError: BaseModel.fuse() got an unexpected keyword argument 'verbose'